# The Trading Game

### Go to the bottom and press play!!! Decide the lowest card, highest card, number of cards, and number of games

In [1]:
import random

In [2]:
# Practice EV

In [9]:
def EV_card(lowest_card, highest_card):
    total = 0
    current = lowest_card
    number_of_cards = (highest_card+1) - lowest_card
    while current <= highest_card:
        total += current
        current += 1
    return total/number_of_cards

In [10]:
def total_EV_init(number_of_cards, card_EV):
    total = number_of_cards*card_EV
    return total

In [17]:
def skewed_mid(true_EV, position, max_absolute_position, max_skew_points):
    """
    Returns a skewed mid based on current position.

    - true_EV: fair value (unskewed mid)
    - position: your current position (negative = short, positive = long)
    - max_absolute_position: the maximum absolute position in the game
    - max_skew_points: how many points you shift the mid at max position
    """
    if max_absolute_position == 0 or max_skew_points == 0:
        return true_EV

    # Normalise position into [-1, 1]
    pos_ratio = position / max_absolute_position  # -1 = max short, +1 = max long

    # short (pos < 0) -> shift mid UP  (positive shift)
    # long  (pos > 0) -> shift mid DOWN (negative shift)
    shift = -pos_ratio * max_skew_points

    return true_EV + shift


In [33]:
import random

# ANSI color codes for pretty output (works in most terminals / Jupyter)
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"

# ---- your existing helpers (unchanged) ----
def EV_card(lowest_card, highest_card):
    total = 0
    current = lowest_card
    number_of_cards = (highest_card+1) - lowest_card
    while current <= highest_card:
        total += current
        current += 1
    return total/number_of_cards

def total_EV_init(number_of_cards, card_EV):
    total = number_of_cards*card_EV
    return total


def run_game(lowest_card, highest_card, number_of_cards, number_of_games,
             max_absolute_position, max_skew_points=0):
    """
    max_skew_points is currently unused (kept for compatibility).
    Skew is enforced via EV-bid vs offer-EV inequalities instead.
    """

    # EV for this deck setup (same for all games)
    card_EV = EV_card(lowest_card, highest_card)
    initial_EV = total_EV_init(number_of_cards, card_EV)

    print("EV per card")
    print(card_EV)
    print("Total EV per game (before flipping any cards)")
    print(initial_EV)
    print()

    # ----- quote width rules (shown once, before any game) -----
    width_rules = {
        10: (1, 1),
        20: (1, 2),
        30: (2, 3),
        40: (3, 4),
        50: (4, 5),
        60: (4, 5),
        70: (5, 6), 
        80: (5, 6),
        90: (5, 7),
        100: (5, 7),
    }

    print("Quote exercise:")
    print("  Max quote size is 50 lots.")
    print("  Width rules (ask - bid):")
    for size, (wmin, wmax) in width_rules.items():
        if wmin == wmax:
            print(f"    {size} lots: width = {wmin} point")
        else:
            print(f"    {size} lots: width = {wmin}–{wmax} points")
    print("  A VALID quote must also satisfy:")
    print("    • EV must lie between your bid and ask (inclusive).")
    print("    • Skew rules:")
    print("        - Short position (<0): EV - bid < offer - EV")
    print("        - Long position  (>0): EV - bid > offer - EV")
    print("        - Flat position  (=0): EV - bid == offer - EV (approximately)")
    print("    • Format: X-Y, with X < Y (e.g. 24-25).")
    print()

    all_games_card_values = []  # list of lists
    all_games_EV_lists = []     # list of lists

    # Overall stats across all games
    total_correct = 0
    total_questions = 0
    total_quote_correct = 0
    total_quote_questions = 0
    stopped_early = False

    # list of possible positions, e.g. max_absolute_position = 50
    # -> [-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50]
    positions = list(range(-max_absolute_position, max_absolute_position + 1, 10))

    # tolerance for "equal" gaps when flat
    skew_tolerance = 0.01

    for game_idx in range(1, number_of_games + 1):

        # create card values for this game
        card_values = [random.randint(lowest_card, highest_card)
                       for _ in range(number_of_cards)]

        print("=" * 50)
        print(f"Start game {game_idx}:")
        print("=" * 50)
        print()

        games_EV_list = [initial_EV]

        # --- masked display of cards ---
        revealed = ["X"] * number_of_cards

        def show_cards(state):
            # prints like: [X, X, 5, X]
            as_str = ", ".join(str(v) for v in state)
            print(f"Card values: [{as_str}]")

        # initial hidden state
        show_cards(revealed)
        print(f"Initial EV: {initial_EV}")
        print()

        prior_sum = 0
        num_correct = 0
        num_questions = 0
        quote_correct = 0
        quote_questions = 0

        for idx, value in enumerate(card_values):
            # ---- position is assigned BEFORE this card is turned over ----
            card_position = random.choice(positions)
            print(f"Position before this card: {card_position}")

            # flip card
            revealed[idx] = value
            prior_sum += value

            show_cards(revealed)

            remaining_cards = number_of_cards - (idx + 1)
            print("Remaining cards:", remaining_cards)

            true_EV = prior_sum + total_EV_init(remaining_cards, card_EV)
            games_EV_list.append(true_EV)

            # --------- LAST CARD BEHAVIOUR: ONLY "MARKET FINISHES" QUESTION ---------
            if remaining_cards == 0:
                final_market = true_EV  # sum of all cards
                while True:
                    finish_str = input("What does the market finish at? (or 'end' to quit): ")
                    if finish_str.lower() == "end":
                        stopped_early = True
                        break
                    try:
                        finish_guess = float(finish_str)
                        break
                    except ValueError:
                        print("Please enter a valid number or 'end'.")

                if stopped_early:
                    break

                if round(finish_guess, 2) == round(final_market, 2):
                    print(f"{GREEN}Correct!{RESET}")
                else:
                    print(f"{RED}False.{RESET} Final market = {round(final_market, 2)}")
                print()
                # End of game after final card
                break

            # --------- NON-LAST CARDS: NORMAL EV + QUOTES FLOW WITH NEW SKEW ---------

            # ---- EV QUIZ PART ----
            while True:
                guess_str = input("Enter your EV guess (or 'end' to quit): ")
                # allow user to end early
                if guess_str.lower() == "end":
                    stopped_early = True
                    break
                try:
                    guess = float(guess_str)
                    break
                except ValueError:
                    print("Please enter a valid number or 'end'.")

            if stopped_early:
                break  # break out of the card loop

            num_questions += 1
            total_questions += 1

            # Compare rounded values (to 2 decimal places)
            if round(guess, 2) == round(true_EV, 2):
                num_correct += 1
                total_correct += 1
                print(f"{GREEN}Correct EV!{RESET}")
            else:
                print(f"{RED}Incorrect EV.{RESET} True EV = {round(true_EV, 2)}")

            print()  # blank line

            # ---- QUOTE EXERCISE: 3 quotes per card, varied lot sizes ----
            # choose 3 distinct lot sizes from [10,20,30,40,50]
            lot_sizes_for_card = random.sample([10, 20, 30, 40, 50, 60, 70, 80, 90, 100], k=3)

            for quote_idx, lot_size in enumerate(lot_sizes_for_card, start=1):
                min_w, max_w = width_rules[lot_size]
                print(f"Quote {quote_idx}/3 for this card: {lot_size} lots.")
                print(f"(Fair EV: {round(true_EV, 2)})")

                while True:
                    quote_str = input("Enter your quote (or 'end' to quit): ")
                    if quote_str.lower() == "end":
                        stopped_early = True
                        break

                    # basic syntax check: X-Y
                    parts = quote_str.replace(" ", "").split("-")
                    if len(parts) != 2:
                        print("Please enter in the form X-Y, e.g. 24-25.")
                        continue
                    try:
                        bid = float(parts[0])
                        ask = float(parts[1])
                    except ValueError:
                        print("Bid and ask must be numbers. Try again.")
                        continue

                    if bid >= ask:
                        print("Bid must be strictly lower than ask. Try again.")
                        continue

                    width = ask - bid
                    quote_questions += 1
                    total_quote_questions += 1

                    # enforce BOTH width rules and EV-inside rule
                    good_width = (min_w <= width <= max_w)
                    ev_inside = (bid <= true_EV <= ask)

                    # --- NEW SKEW CHECKS based on card_position ---
                    # gaps from EV to each side
                    gap_bid = true_EV - bid     # EV - bid
                    gap_ask = ask - true_EV     # offer - EV

                    if card_position < 0:
                        # short: EV - bid < offer - EV
                        skew_ok = (gap_bid + skew_tolerance < gap_ask)
                    elif card_position > 0:
                        # long: EV - bid > offer - EV
                        skew_ok = (gap_bid > gap_ask + skew_tolerance)
                    else:
                        # flat: EV - bid == offer - EV (approximately)
                        skew_ok = abs(gap_bid - gap_ask) <= skew_tolerance

                    # ---- VALIDITY INCLUDING SKEW ----
                    if good_width and ev_inside and skew_ok:
                        quote_correct += 1
                        total_quote_correct += 1
                        print(f"{GREEN}Quote is valid!{RESET}")
                    else:
                        msg_parts = []
                        if not good_width:
                            msg_parts.append(
                                f"width {width:.2f} not in [{min_w}, {max_w}]"
                            )
                        if not ev_inside:
                            msg_parts.append(
                                f"EV {round(true_EV, 2)} not between {bid} and {ask}"
                            )
                        if not skew_ok:
                            if card_position < 0:
                                msg_parts.append(
                                    f"skew wrong for short: need EV-bid < offer-EV "
                                    f"(got {gap_bid:.2f} vs {gap_ask:.2f})"
                                )
                            elif card_position > 0:
                                msg_parts.append(
                                    f"skew wrong for long: need EV-bid > offer-EV "
                                    f"(got {gap_bid:.2f} vs {gap_ask:.2f})"
                                )
                            else:
                                msg_parts.append(
                                    f"skew wrong for flat: need EV-bid ≈ offer-EV "
                                    f"(got {gap_bid:.2f} vs {gap_ask:.2f})"
                                )
                        detail = "; ".join(msg_parts)
                        print(f"{RED}Invalid quote.{RESET} ({detail})")

                    # extra skew feedback
                    print(f"EV - bid = {gap_bid:.2f}, offer - EV = {gap_ask:.2f}")
                    print()
                    break  # move to next quote after one validly-parsed quote

                if stopped_early:
                    break  # break out of quote loop

            if stopped_early:
                break  # out of card loop

        # store results for this game
        all_games_card_values.append(card_values)
        all_games_EV_lists.append(games_EV_list)

        # per-game accuracy
        game_accuracy = (num_correct / num_questions * 100) if num_questions > 0 else 0.0
        quote_accuracy = (quote_correct / quote_questions * 100) if quote_questions > 0 else 0.0

        print("=" * 50)
        print(f"End game {game_idx}:")
        print("=" * 50)
        print("EV progression:", games_EV_list)
        print(f"EV questions this game:     {num_questions}")
        print(f"EV correct this game:       {num_correct}")
        print(f"EV accuracy this game:      {game_accuracy:.1f}%")
        print(f"Quote questions this game:  {quote_questions}")
        print(f"Valid quotes this game:     {quote_correct}")
        print(f"Quote accuracy this game:   {quote_accuracy:.1f}%")
        print()

        if stopped_early:
            break  # break out of the games loop too

    # Overall accuracy (across all questions actually answered)
    print("=" * 50)
    if stopped_early:
        print("Quiz ended early by user.")
    else:
        print("All games completed.")
    print("=" * 50)

    if total_questions > 0:
        overall_accuracy = total_correct / total_questions * 100
    else:
        overall_accuracy = 0.0

    if total_quote_questions > 0:
        overall_quote_accuracy = total_quote_correct / total_quote_questions * 100
    else:
        overall_quote_accuracy = 0.0

    print(f"Total EV questions answered:   {total_questions}")
    print(f"Total EV correct answers:      {total_correct}")
    print(f"Overall EV accuracy:           {overall_accuracy:.1f}%")
    print(f"Total quote questions:         {total_quote_questions}")
    print(f"Total valid quotes:            {total_quote_correct}")
    print(f"Overall quote accuracy:        {overall_quote_accuracy:.1f}%")
    print()

    return all_games_card_values, all_games_EV_lists


In [34]:
run_game(
    lowest_card=1,
    highest_card=9,
    number_of_cards=5,
    number_of_games=2,
    max_absolute_position=100,
    max_skew_points=2,  # tweak this to make skew more/less aggressive
)

EV per card
5.0
Total EV per game (before flipping any cards)
25.0

Quote exercise:
  Max quote size is 50 lots.
  Width rules (ask - bid):
    10 lots: width = 1 point
    20 lots: width = 1–2 points
    30 lots: width = 2–3 points
    40 lots: width = 3–4 points
    50 lots: width = 4–5 points
    60 lots: width = 4–5 points
    70 lots: width = 5–6 points
    80 lots: width = 5–6 points
    90 lots: width = 5–7 points
    100 lots: width = 5–7 points
  A VALID quote must also satisfy:
    • EV must lie between your bid and ask (inclusive).
    • Skew rules:
        - Short position (<0): EV - bid < offer - EV
        - Long position  (>0): EV - bid > offer - EV
        - Flat position  (=0): EV - bid == offer - EV (approximately)
    • Format: X-Y, with X < Y (e.g. 24-25).

Start game 1:

Card values: [X, X, X, X, X]
Initial EV: 25.0

Position before this card: 60
Card values: [4, X, X, X, X]
Remaining cards: 4


Enter your EV guess (or 'end' to quit):  24


Correct EV!

Quote 1/3 for this card: 50 lots.
(Fair EV: 24.0)


Enter your quote (or 'end' to quit):  20-25


Quote is valid!
EV - bid = 4.00, offer - EV = 1.00

Quote 2/3 for this card: 80 lots.
(Fair EV: 24.0)


...

Enter your EV guess (or 'end' to quit):  17


Correct EV!

Quote 1/3 for this card: 40 lots.
(Fair EV: 17.0)


Enter your quote (or 'end' to quit):  end


End game 1:
EV progression: [25.0, 24.0, 21.0, 17.0]
EV questions this game:     3
EV correct this game:       3
EV accuracy this game:      100.0%
Quote questions this game:  6
Valid quotes this game:     4
Quote accuracy this game:   66.7%

Quiz ended early by user.
Total EV questions answered:   3
Total EV correct answers:      3
Overall EV accuracy:           100.0%
Total quote questions:         6
Total valid quotes:            4
Overall quote accuracy:        66.7%



([[4, 2, 1, 2, 6]], [[25.0, 24.0, 21.0, 17.0]])